In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import pysindy as ps
import math
from math import pi

Variables globales:

In [ ]:
def coefs_ecs(p_c):
    coefs = {
        'A': p_c['T']/p_c['m'],
        'B': (pi*p_c['rho']*p_c['cd0']*(p_c['d']**2))/(8*p_c['m']),
        'C': p_c['g'],
        'D': (1-pi*p_c['rho']*(p_c['d']**3)*p_c['cntp']/(16*p_c['m']))/(1+pi*p_c['rho']*(p_c['d']**3)*p_c['cnap']/(16*p_c['m'])),
        'E': (pi*p_c['rho']*(p_c['d']**2)*p_c['cna']/(16*p_c['m']))/(1+pi*p_c['rho']*(p_c['d']**3)*p_c['cnap']/(16*p_c['m'])),
        'F': (p_c['T'])/(p_c['m']+pi*p_c['rho']*(p_c['d']**3)*p_c['cnap']/(16)),
        'G': p_c['g']/(1+pi*p_c['rho']*(p_c['d']**3)*p_c['cnap']/(16*p_c['m'])),
        'H': (pi*p_c['rho']*(p_c['d']**3)*p_c['cma'])/(8*p_c['Iy']),
        'I': (pi*p_c['rho']*(p_c['d']**4)*p_c['cmtp'])/(16*p_c['Iy']),
        'J': (pi*p_c['rho']*(p_c['d']**4)*p_c['cmap'])/(16*p_c['Iy'])
    }
    return coefs

In [ ]:
#que son los coeficiente del cn??????????? 
paso_tiempo=0.001
max_time = 30.0

#parametros fisicos del cohete:
param_cohe = {
    'T': 29160, #empuje
    'm': 77.4, #masa
    'rho': 1.225, #densidad
    'd': 0.14, #diametro
    'cntp': 67.934021, #cntheta'
    'cnap': 50.813892,#cn alfa'
    'cna': 0.415000,#cnalfa
    'Iy': 35.63,#inercia en el eje y
    'cma': -35.581600,#cm alfa
    'cmtp': 60.934021,#cm theta'
    'cmap': -225.731250,#cm alfa'
    'cd0': 0.270000,#fallo en el pdf???????????????????? 
    'g': 9.81, #gravedad
}

#coeficientes de las ecuaciones del cohete 2D:
coefs = coefs_ecs(param_cohe)

#valores iniciales:
inicial_values = {
    'x': 1.,
    'y': 1.,
    'vel': 0.1,
    'alfa': 0,
    'theta': 0, #40 grados
    'theta_dot': 0 
}
 #x,y,v, alfa, theta, theta'

Las incógnitas son theta, alfa, V, theta prima y alfa prima. Ecuaciones a modelar:

In [ ]:
def eqs(t_train, x0_train, a=coefs['A'], b=coefs['B'], c=coefs['C'], d=coefs['D'], e=coefs['E'], f=coefs['F'], 
        g=coefs['G'],h=coefs['H'], i=coefs['I'], jota=coefs['J']):
    
    x0, y0, v0, alfa0, theta0, u0 = x0_train
    
    vel_dot = a*math.cos(alfa0) - b*v0**2 - c*math.sin(theta0-alfa0)
    alfa_dot = d*u0 - e*v0*alfa0 - f*math.sin(alfa0)*1/v0 + g*math.cos(theta0 - alfa0)*1/v0
    theta_dot_dot = h*alfa0*v0**2 + i*v0*u0 + jota*v0*alfa_dot
    
    x_dot = v0*math.cos(theta0-alfa0)
    y_dot = v0*math.sin(theta0-alfa0)
    
    theta_dot = u0
    return x_dot, y_dot, vel_dot, alfa_dot, theta_dot, theta_dot_dot

In [ ]:
#x0_train = [x, y, v, alfa, theta, theta']
def calculate_data(t_max=max_time, dt=paso_tiempo, 
                    a=coefs['A'], b=coefs['B'], c=coefs['C'], d=coefs['D'], e=coefs['E'], f=coefs['F'], 
                    g=coefs['G'], h=coefs['H'], i=coefs['I'], jota=coefs['J'],
                    x0_train=[inicial_values['x'], inicial_values['y'], inicial_values['vel'], 
                              inicial_values['alfa'], inicial_values['theta'], inicial_values['theta_dot']]):
    
    t_train = np.arange(0.0, t_max, dt)
    
    #sol = np.ones((t_train.size, 5))
    print(t_train[0], t_train[-1])
    
    x_sol = solve_ivp(eqs, (t_train[0], t_train[-1]), x0_train, args=(a,b,c,d,e,f,g,h,i,jota), t_eval=t_train) 
    #[x, y, v, alfa, theta, theta']
    print(x_sol)
    
    x_dot_train_measured = np.ones((t_train.size, 6)) #contiene, x',y',v',alfa',theta''
    
   
    fig = plt.figure()
    #ax = fig.gca(projection='2d')

    plt.plot(x_sol.t, x_sol.y.T[:, 0], lw=0.5)
    plt.xlabel("tiempo")
    plt.ylabel("posicion en x")
    plt.title("Trayectoria en x")

    plt.show()
    
    fig = plt.figure()
    #ax = fig.gca(projection='2d')

    plt.plot(x_sol.t, x_sol.y.T[:, 1], lw=0.5)
    plt.xlabel("tiempo")
    plt.ylabel("posicion y")
    plt.title("Trayectoria en y")

    plt.show()
    
    fig = plt.figure()
    #ax = fig.gca(projection='2d')

    plt.plot(x_sol.y.T[:, 0], x_sol.y.T[:, 1], lw=0.5)
    plt.xlabel("tiempo")
    plt.ylabel("posicion y")
    plt.title("posicion x frente a y")

    plt.show()
   
    plt.plot(x_sol.t, x_sol.y.T)
    plt.legend(["x", "y", "v", "alfa", "theta", "theta'"])
    plt.show()
    print(t_train.size, x_sol.y.T.shape)
    for i in range(t_train.size):
        x_dot_train_measured[i, :] = eqs(t_train, x_sol.y.T[i, :], a,b,c,d,f,g,h,i,jota)

    
    
    
    return x_sol.y.T, x_dot_train_measured

Defino mi propia librería de funciones:

In [ ]:
def custom_library(poly_order=2):
    library_functions = [
        lambda theta, alfa : np.cos(theta-alfa),
        lambda theta, alfa : np.sin(theta-alfa),
        lambda alfa : np.sin(alfa),
        lambda alfa : np.cos(alfa),
        #lambda v: v**2,
        lambda v, alfa: np.sin(alfa)/v 
    ]
    library_function_names = [
        lambda theta, alfa : 'cos(' + theta + '-' + alfa +')',
        lambda theta, alfa : 'sen(' + theta + '-' + alfa +')',
        lambda alfa : 'sen('  + alfa +')',
        lambda alfa : 'cos('  + alfa +')',
        #lambda v: v + '^2',
        lambda v, alfa: 'sen(' + alfa + ')/' + v 
    ]
    library = ps.CustomLibrary(
                        library_functions=library_functions, 
                        function_names=library_function_names) + ps.PolynomialLibrary(degree=poly_order)
    return library

In [ ]:
def custom_library2(poly_order=2):
    library_functions = [
        lambda theta, alfa : np.cos(theta-alfa),
        #lambda theta, alfa, v : v*np.cos(theta-alfa),
        lambda theta, alfa, v : np.cos(theta-alfa)*1./v,
        lambda theta, alfa : np.sin(theta-alfa),
        #lambda theta, alfa, v : v*np.sin(theta-alfa),
        lambda alfa : np.sin(alfa),
        lambda alfa, v : np.sin(alfa)*1./v,
        lambda alfa : np.cos(alfa),
        lambda v: 1./v,
        
    ]
    library_function_names = [
        lambda theta, alfa : 'cos(' + theta + '-' + alfa +')',
        #lambda theta, alfa, v : v + '*' + 'cos(' + theta + '-' + alfa +')',
        lambda theta, alfa, v : 'cos(' + theta + '-' + alfa +')/' + v,
        lambda theta, alfa : 'sen(' + theta + '-' + alfa +')',
        #lambda theta, alfa, v : v + '*' + 'sen(' + theta + '-' + alfa +')',
        lambda alfa : 'sen('  + alfa +')',
        lambda alfa, v : 'sen('  + alfa +')/' + v,
        lambda alfa : 'cos('  + alfa +')',
        lambda v :  '1/' + v,
        
    ]
    library = ps.CustomLibrary(
                        library_functions=library_functions, 
                        function_names=library_function_names) + ps.PolynomialLibrary(degree=poly_order)
    return library

In [ ]:
def sindy_model(x_train, x_dot_train, threshold = 0.2, terminos_ctes=False, dt=paso_tiempo):
    # Fit the model, me creo el modelo dinámico 

    #pysindy calcula las ecs que gobierna el movimiento
    model = ps.SINDy(
        optimizer=ps.STLSQ(threshold=threshold, normalize=True,  fit_intercept=terminos_ctes),
        feature_library=custom_library(),#interaction_only=terminos_ctes,
                                                         
        feature_names=['x', 'y', 'v', 'alfa', 'theta', "theta'"],
    )

    #la doc dice que en t hay que poner el time step
    model.fit(x_train, t=dt, x_dot=x_dot_train, quiet=True)

    #print(model.equations(precision=52))
    model.print(precision=4)
    #print(model.coefficients())
    return model

Genero los datos:

In [ ]:
x_train, x_dot_train = calculate_data()
print(x_train)
print(x_train.shape)



In [ ]:
model = sindy_model(x_train, x_dot_train)

In [ ]:
a = np.arange(0.0, max_time, paso_tiempo)
print(a.shape)

## Trayectoria:

In [ ]:
print(coefs)